# Data pre-processing notebook
### Dropping non-songs, encoding categorical variables

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import category_encoders as ce

df = pd.read_csv("../../data/tracks.csv")
# df.head()

Data cleanup

In [4]:
!pip install category_encoders

In [2]:
df.columns

Index(['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists',
       'id_artists', 'release_date', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')

In [3]:

def new_clean_data(df):
   # Drop the id column, name, release_date
   x = df.copy()
   drop_cols = ['id', 'name', 'release_date', 'id_artists', 'artists']
   x = x.drop(labels=drop_cols, axis=1)
    
   # Drop zero tempo songs
   x = x[x['tempo']!=0]

   # Create categorical variables: 
   #     splits each feature into 5 categories (bins)
   #     key, explicit not included
   to_cat = ['popularity', 'duration_ms', 'danceability', 'energy',
      'loudness', 'speechiness', 'acousticness', 'instrumentalness',
      'liveness', 'valence', 'tempo', 'time_signature']
   for col in to_cat:
      x[col] = pd.cut(x[col], bins=5, labels=False)
 
   #  One-hot encoding of all features
   cols = ['popularity', 'duration_ms', 'danceability', 'energy',
      'loudness', 'speechiness', 'acousticness', 'instrumentalness',
      'liveness', 'valence', 'tempo', 'time_signature', 'key', 'explicit']
   ohe = ce.one_hot.OneHotEncoder(cols=cols)
   x = ohe.fit_transform(x)
    
   return x


In [4]:
new_clean_df = new_clean_data(df)
new_clean_df.head()

/Users/Carl/miniconda3/envs/U4-S2-NN/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,popularity_1,popularity_2,popularity_3,popularity_4,popularity_5,duration_ms_1,duration_ms_2,duration_ms_3,duration_ms_4,duration_ms_5,...,valence_5,tempo_1,tempo_2,tempo_3,tempo_4,tempo_5,time_signature_1,time_signature_2,time_signature_3,time_signature_4
0,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [9]:
new_clean_df.describe()
new_clean_df.columns

Index(['popularity_1', 'popularity_2', 'popularity_3', 'popularity_4',
       'popularity_5', 'duration_ms_1', 'duration_ms_2', 'duration_ms_3',
       'duration_ms_4', 'duration_ms_5', 'explicit_1', 'explicit_2',
       'danceability_1', 'danceability_2', 'danceability_3', 'danceability_4',
       'danceability_5', 'energy_1', 'energy_2', 'energy_3', 'energy_4',
       'energy_5', 'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6',
       'key_7', 'key_8', 'key_9', 'key_10', 'key_11', 'key_12', 'loudness_1',
       'loudness_2', 'loudness_3', 'loudness_4', 'loudness_5', 'mode',
       'speechiness_1', 'speechiness_2', 'speechiness_3', 'speechiness_4',
       'speechiness_5', 'acousticness_1', 'acousticness_2', 'acousticness_3',
       'acousticness_4', 'acousticness_5', 'instrumentalness_1',
       'instrumentalness_2', 'instrumentalness_3', 'instrumentalness_4',
       'instrumentalness_5', 'liveness_1', 'liveness_2', 'liveness_3',
       'liveness_4', 'liveness_5', 'valence_1', 'v

In [39]:
new_clean_df.to_csv('../../data/new_songs_cleaned.csv')